In [2]:
import pandas as pd 

data = pd.read_csv("movies_metadata.csv")
data.head() 
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
data=data.head(20000)
data['overview'].isnull().sum()

135

In [4]:
data['overview']=data['overview'].fillna('')
data['overview'].isnull().sum()

0

In [5]:
#TFIDF 구하기
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tfidf=TfidfVectorizer(stop_words='english')

In [7]:
tfidfMatrix=tfidf.fit_transform(data['overview'])

In [8]:
tfidfMatrix.shape

(20000, 47487)

In [9]:
tfidfMatrix

<20000x47487 sparse matrix of type '<class 'numpy.float64'>'
	with 535909 stored elements in Compressed Sparse Row format>

In [10]:
#코사인 유사도(linear_kernel)를 구하는 공식
from sklearn.metrics.pairwise import linear_kernel

In [11]:
cos_sim=linear_kernel(tfidfMatrix, tfidfMatrix)

In [12]:
cos_sim.shape
cos_sim

array([[1.        , 0.01575748, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01575748, 1.        , 0.04907345, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.04907345, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.08375766],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08375766, 0.        ,
        1.        ]])

In [13]:
1)Toy Story 영화 제목 입력 받음
2)Toy Story 코사인 유사도가 가장 높은 영화 제목을 추출

SyntaxError: invalid syntax (<ipython-input-13-39e07d821c4e>, line 1)

In [14]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [15]:
data['title'].duplicated().sum() # duplicated = 중복된 값이 있는지 없는지를 확인 할 수 있다. 

901

In [16]:
df = pd.DataFrame({'k1':['a', 'b', 'b', 'c', 'b', 'c']})
df.duplicated(['k1'])
df.drop_duplicates()

,k1
0,a
1,b
3,c


In [17]:
data['title'].duplicated().sum()
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [18]:
indices['Batman']

Batman     585
Batman    8603
dtype: int64

In [19]:
import numpy as np 

In [20]:
def getRecommandation(title, cosine_sim=cos_sim): # cos_sim 코사인 유사도를 
    #title에 저장된 영화 제못으로 코사인 유사도가 가장 높은 10편의 영화를 추출하겠다. 
    idx = indices[title]
    # cosine_sim[idx]
    # print(len(cosine_sim[idx]))
    # print(np.max(cosine_sim[idx]))
    simScores=(list(enumerate(cosine_sim[idx])))
    # print(sorted(simScores, key =lambda x:x[1] ,reverse=True)) #x에 전달된 tuple의 1번 index에 해당되는 값으로 정렬 
    simScores=sorted(simScores, key =lambda x:x[1] ,reverse=True)
    simScores[1:21]
    movieidx=[i[0] for i in simScores]
    return data['title'].iloc[movieidx]
    #simScores의 유사도 값을 기준으로 내림차순 정렬 
    # print(title)

In [21]:
getRecommandation("Toy Story")

0                                                Toy Story
15348                                          Toy Story 3
2997                                           Toy Story 2
10301                               The 40 Year Old Virgin
8327                                             The Champ
                               ...                        
19994                               Violeta Went to Heaven
19995                                            Rebellion
19997                                      Two in the Wave
19998    Lotte Reiniger: Homage to the Inventor of the ...
19999    RKO Production 601: The Making of 'Kong, the E...
Name: title, Length: 20000, dtype: object

In [22]:
#영화 리뷰 분류(이진 분류)
from keras.datasets import imdb

In [23]:
imdb.load_data()
(traindata, trainlabels),(testdata, testlabels)=imdb.load_data(num_words=10000) #자주 사용되어 지는 단어 10000개만 추려서 사용하겠다. 0~9999 까지 

In [24]:
traindata.shape #25000개 훈련 데이터(리뷰)
# traindata
trainlabels.shape #25000 : 0=부정, 1=긍정 
# trainlabels
testdata.shape #25000
# testdata

(25000,)

In [25]:
traindata[0] # 첫번째 관객이 남긴 리뷰에 대한 수치 관계를 표현한 것 

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [26]:
for seq in traindata:
    print(max(seq)) # max를 적용시키면, 각 댓글에 남긴 단어중 index번호가 가장 큰 것 만 출력 

7486
9837
6905
9941
7224
7982
9363
9820
7612
8419
8914
7412
9667
9106
6640
9671
9263
9935
8742
9580
9859
8793
7609
9462
9075
8614
8887
5898
9637
9212
9771
9209
8257
7778
9877
9753
7015
9902
8427
8867
8406
9579
9529
7056
7023
9922
5811
9124
9910
8795
7799
7146
9978
7999
9565
9731
9095
9888
5247
9655
4852
9939
9018
9885
7239
9636
9389
9561
9673
9147
9956
9448
8520
9360
9765
9025
9928
8352
9960
7320
8318
7881
9715
9851
9266
9055
8894
9719
9271
7311
8791
9918
9486
9867
9283
7707
9133
9055
9978
8013
8448
8700
9906
7517
9962
6266
6903
9815
9976
9273
9413
8931
8631
9685
9419
9964
7361
9052
9482
9643
9865
9819
7181
2775
9709
9977
8916
4431
9700
6532
8836
9926
6781
7614
5418
8881
9841
9615
9072
9831
6891
8827
9019
9583
8776
8534
9853
8368
8931
9830
8084
5991
7901
9671
6817
9937
7960
7676
8653
6866
6325
9737
9771
5636
8557
7371
9995
8689
9294
8893
8462
6975
9665
8802
7608
9933
9738
9418
7542
7073
8448
9570
9916
9315
7806
9901
9388
9723
9049
9985
8427
8376
9985
9584
9426
9840
9208
8648
9675
9608


In [27]:
maxseq=[max(seq) for seq in traindata]
maxseq # 위와 같다
max(maxseq) # 전체의 최대값 

9999

In [28]:
word_index=imdb.get_word_index() #dictionary구조 (단어, 인덱스)
word_index
len(word_index)
word_index.items() #tuple형식으로 변한다 ('fawn', 34701)
dict([(key, value) for (key, value) in word_index.items()]) # 다시 dictionary구조, 원래의 구조과 똑같다 
dict([(value, key) for (key, value) in word_index.items()]) # 이번에는 index가 먼저 나오고, key(단어) 가 나중에 나온다. 

# (인덱스, 단어)
reverse_word_index = dict([(value, key) 
                        for (key, value) 
                        in word_index.items()])

In [29]:
reverse_word_index
traindata[0]
reverse_word_index[1]
# 또는
reverse_word_index.get(1)

'the'

In [30]:
print(reverse_word_index.get(1000000, "?")) # 만약 index가 100000이 존재 하지 않는 다면, "?"로 대체해라 

#0,1,2 는 "패딩", "문서시작", "사전에 없음(oov)"
#위의 인덱스로 약속되어 사용되고 있음. 따라서 위의 3개는 제외 
#위의 3개는 "?" 문자로 대체 

?


In [31]:
traindata[0]

decodedReview=" ".join(reverse_word_index.get(i-3,"?")
                        for i in traindata[0])
# index번호에서 3을 뺀 것을 가져오겠다. 만약 i가 2여서 i-3=-1이 된다면 "?로 대체하겠다"

In [32]:
decodedReview

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [33]:
"""
패딩: 문장의 길이가 다름 ->문장의 길이를 동일하게 해줌 
신경망 모델을 만들기 위해서는 먼저 패딩을 수행하여
모든 문장의 길이를 동일하게 해줘야 함. 
"""

'\n패딩: 문장의 길이가 다름 ->문장의 길이를 동일하게 해줌 \n신경망 모델을 만들기 위해서는 먼저 패딩을 수행하여\n모든 문장의 길이를 동일하게 해줘야 함. \n'

In [34]:
sentences = [['barber', 'person'], 
            ['barber', 'good', 'person'], ['barber', 'huge', 'person'], 
            ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], 
            ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], 
            ['barber', 'went', 'huge', 'mountain']]

In [35]:
#정수 인코딩
from tensorflow.keras.preprocessing.text import Tokenizer


In [36]:
tok=Tokenizer()
tok.fit_on_texts(sentences)
#빈도수를 기준으로 단어 집합을 생성해주는 함수 

In [37]:
# tok.index_word
encoded=tok.texts_to_sequences(sentences)
encoded 
# 모든 문장들의 길이를 동일하게 해줘야 한다. 
# encoded에 저장된 문장들 중에서 가장 긴 문장의 길이를 구해보자 

[len(item) for item in encoded] #문장의 길이를 구해보자 
maxLen=max(len(item) for item in encoded) # 최대 문장의 길이 7

In [38]:
for item in encoded:
    while len(item)<maxLen:
        item.append(0)
padded_np = np.array(encoded)
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

In [39]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [40]:
sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [41]:
#케라스에서 제로 패딩 
encoded=tok.texts_to_sequences(sentences)
padded=pad_sequences(encoded, padding='post') #반대로 나온것을 뒤집어서 출력 
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

In [42]:
def vectorize_sequences(sequences, dimension=10000):
    print(len(sequences)) # 25000행, 10000열 구성
    results = np.zeros((len(sequences), dimension))
    for i, seq in enumerate(sequences):
        # print(i, "번 리뷰", seq)
        results[i, seq]=1
    return results
xtrain = vectorize_sequences(traindata)
xtest = vectorize_sequences(testdata)

vectorize_sequences(traindata)

25000
25000
25000


array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [45]:
xtrain[0] # 첫번째 리뷰에 등장하는 단어들 -> 그 단어를 수치화 해서 가지고 있었음 
xtrain[0].shape #10000

(10000,)

In [48]:
ytrain=np.asarray(trainlabels).astype('float32') # array로 변환하고 타입 변환 
ytrain

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [44]:
np.zeros((3,2))

array([[0., 0.],
       [0., 0.],
       [0., 0.]])